In [1]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
Setup complete ✅ (10 CPUs, 16.0 GB RAM, 411.1/460.4 GB disk)


In [2]:
import os
import random
import shutil
from tqdm.notebook import tqdm
import cv2
import glob
import json
import numpy as np
import matplotlib.pyplot as plt

In [3]:
origin_data_root = '/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation'
cls_list = ['Scratched', 'Breakage', 'Separated', 'Crushed']
data_root = '/Users/kimhongseok/cv_79_projects/part2/26/data'

train_root = f'{data_root}/train'
valid_root = f'{data_root}/valid'
test_root = f'{data_root}/test'

for folder in [train_root, valid_root, test_root]:
    if not os.path.exists(folder):
        os.makedirs(folder)

    for s in ['images', 'labels']:
        s_folder = os.path.join(folder, s)
        if not os.path.exists(s_folder):
            os.makedirs(s_folder)

In [4]:
def json_to_yolo_polygon(polygon, w, h):
    yolo_list = []
    for p in polygon:
        yolo_list.append(p[0]/w)
        yolo_list.append(p[1]/h)
    return ' '.join([str(x) for x in yolo_list])

In [5]:
file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/2.라벨링데이터/damage_part/*.json')
random.seed(2024)
random.shuffle(file_list)
print(len(file_list))

17248


In [7]:
# 라벨링 형태 변환
mask_name = file_list[0].split('/')[-1].replace('json', 'txt')
result = []

with open(file_list[0], 'r') as json_file:
    data = json.load(json_file)
    h = data['images']['height']
    w = data['images']['width']
    for ann in data['annotations']:
        label = ann['damage']
        if label in cls_list:
            polygon_coor = ann['segmentation'][0][0][:-1]
            coor_string = json_to_yolo_polygon(polygon_coor, w, h)
            yolo_string = f'{cls_list.index(label)} {coor_string}'
            result.append(yolo_string)

print(result)
print(len(result))

['2 0.53375 0.4166666666666667 0.51375 0.6066666666666667 0.4975 0.725 0.48 0.8083333333333333 0.46625 0.8583333333333333 0.45875 0.8033333333333333 0.46 0.75 0.465 0.705 0.48 0.62 0.47625 0.5716666666666667 0.5075 0.4666666666666667', '3 0.45 0.7216666666666667 0.43875 0.6583333333333333 0.455 0.6066666666666667 0.4875 0.54 0.4975 0.5166666666666667 0.5925 0.5683333333333334 0.66875 0.5333333333333333 0.67125 0.5833333333333334 0.58375 0.6266666666666667 0.525 0.66', '0 0.41 0.6566666666666666 0.4475 0.615 0.47375 0.5866666666666667 0.4875 0.52 0.52375 0.46166666666666667 0.5375 0.5566666666666666 0.66625 0.515 0.6875 0.5066666666666667 0.7025 0.5516666666666666 0.6475 0.5933333333333334 0.6575 0.6316666666666667 0.56875 0.6283333333333333 0.52375 0.6783333333333333 0.44125 0.7083333333333334', '0 0.74625 0.36333333333333334 0.75 0.3283333333333333 0.81125 0.3433333333333333 0.815 0.37333333333333335']
4


In [10]:
if not os.path.exists('/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/labels'):
    os.makedirs('/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/labels')

tbar = tqdm(file_list[:3000])
for file in tbar:
    result = []
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        h = data['images']['height']
        w = data['images']['width']
        for ann in data['annotations']:
            damage = ann['damage']
            if damage in cls_list:
                polygon_coor = ann['segmentation'][0][0][:-1]
                polygon_string = json_to_yolo_polygon(polygon_coor, w, h)
                yolo_string = f'{cls_list.index(damage)} {polygon_string}'
                result.append(yolo_string)
    if result:
        tmp = file.split('/')[-1].replace('json', 'txt')
        save_path = os.path.join('/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/labels', tmp)
        with open(save_path, 'w', encoding='utf-8') as f:
            f.write('\n'.join(result))

  0%|          | 0/3000 [00:00<?, ?it/s]

In [11]:
random.seed(2024)
file_list = glob.glob('/Users/kimhongseok/cv_79_projects/part2/26/origindata/labels/*.txt')
random.shuffle(file_list)

test_ratio = 0.1
num_file = len(file_list)

test_list = file_list[:int(num_file*test_ratio)]
valid_list = file_list[int(num_file*test_ratio):int(num_file*test_ratio)*2]
train_list = file_list[int(num_file*test_ratio)*2:]

for i in test_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{test_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/1.원천데이터/damage_part/{img_name}'
    shutil.copyfile(img_path, f'{test_root}/images/{img_name}')

for i in valid_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{valid_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/1.원천데이터/damage_part/{img_name}'
    shutil.copyfile(img_path, f'{valid_root}/images/{img_name}')

for i in train_list:
    label_name = i.split('/')[-1]
    shutil.copyfile(i, f'{train_root}/labels/{label_name}')
    img_name = i.split('/')[-1].replace('txt', 'jpg')
    img_path = f'/Users/kimhongseok/cv_79_projects/part2/26/origindata/01.데이터/2.Validation/1.원천데이터/damage_part/{img_name}'
    shutil.copyfile(img_path, f'{train_root}/images/{img_name}')

# config file

In [13]:
import yaml

data = dict()
data['train'] = train_root
data['val'] = valid_root
data['test'] = test_root
data['nc'] = len(cls_list)
data['names'] = cls_list

with open(f'/Users/kimhongseok/cv_79_projects/part2/26/car_damage.yaml', 'w') as f:
    yaml.dump(data, f)

# training

In [12]:
import ultralytics
from ultralytics import YOLO

%cd /Users/kimhongseok/cv_79_projects/part2/26

/Users/kimhongseok/cv_79_projects/part2/26


In [14]:
model = YOLO('yolov8s-seg.yaml') # seg를 위해 seg 모델을 가져온다.
results = model.train(data='car_damage.yaml', epochs=1, batch=16, patience=30, name='test')


New https://pypi.org/project/ultralytics/8.2.101 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.66 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
engine/trainer: task=segment, mode=train, model=yolov8s-seg.yaml, data=car_damage.yaml, epochs=1, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=test, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

train: Scanning /Users/kimhongseok/cv_79_projects/part2/26/data/train/labels... 2338 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2338/2338 [00:00<00:00, 2917.38it/s]

train: WARNING ⚠️ /Users/kimhongseok/cv_79_projects/part2/26/data/train/images/0243852_as-3453103.jpg: 1 duplicate labels removed


train: New cache created: /Users/kimhongseok/cv_79_projects/part2/26/data/train/labels.cache


INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.16 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


...

val: Scanning /Users/kimhongseok/cv_79_projects/part2/26/data/valid/labels... 292 images, 0 backgrounds, 0 corrupt: 100%|██████████| 292/292 [00:00<00:00, 3123.46it/s]

val: New cache created: /Users/kimhongseok/cv_79_projects/part2/26/data/valid/labels.cache


Plotting labels to runs/segment/test/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/segment/test
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


        1/1         0G      3.903      12.82      4.904      4.357         88        640:   1%|          | 1/147 [00:24<1:00:17, 24.78s/it]


KeyboardInterrupt: 